In this notebook, we'll look at reading in raw data files and working with them. We will read in a sample text file and do a quick frequency analysis on it. Once we have that we'll try to build an application to generate an something that "looks like" English. The main points here are

  * Learning to work with external data sources
  * Learning how to quickly explore your data
  * Cleaning your data
  * Performing an analysis
  * Making models and predictions
  
Along the way we'll take a basic look at some concepts from information theory such as probablilty and information entropy.

In [ ]:
# First we start with some boilerplate. 
# Some combination of these imports will appear in almost all 
# of your notebooks or python code

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from collections import Counter

import nltk
import re

## Source Text

I've stolen most of this analysis shamelessly from a course by Benjamin Schumacher [The Science of Information: From Language to Black Holes](https://thegreatcourses.org).

In [ ]:
with open('./ReturnOfSherlockHolmes.txt', 'r') as f:
    rawTxt = f.read()

Let's take a quick look to make sure we have the right thing in sourceTxt. The sourceTxt[:100] syntax should give us the first 100 characters of the file.

In [ ]:
print(rawTxt[:2000])

OK, that pretty good, but let's throw away the header. We can do this by using Python's string methods to search for the String `I.--The Adventure of the Empty House` and trimming there.

In [ ]:
bookTxt = rawTxt[rawTxt.find('I.--The Adventure of the Empty House.'):]
print(bookTxt[:256])

That looks good. Now we want to find the occurences of each token that appears in the text. We could write some loops to do this, but with Python someone has usually already thought of this for you. The collections module, deals with collections of things and it has a Counter object which will slurp up our `bookTxt` and count each item. s

In [ ]:
c = Counter(bookTxt)
c

That looks good, but there here are a few things we want to fix. We only want to deal with the letter frequency so we want 'x' and 'X' to be considered as the same item. Python strings have a `lower` method which will translate everything to lowercase. Next, there are some special characters to deal with. We can replace newlines with a space without changing meaning, but for the other characters (e.g. `&, %, ;`)we will just remove them as well to keep things simple. In summary we want to retain `[a-z ]`. Again hitting tab completion on the string methods turns up a `translate` method which allows us to make replacements, we can use this to replace each of the special characters with the special `None` object. The next expression is pretty complicated, but just start on the inside and work your way out to see what it does:


In [ ]:
letters = [' ', ',', '.', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h',
          'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p',
          'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 
          'y', 'z']

#bookTxt = bookTxt.replace('\n', ' ').lower().translate(
#    {ord(i): None for i in c.keys() if i not in 'abcdefghijklmnopqrstuvwxyz ,.'}
#)
bookTxt = bookTxt.replace('\n', ' ').lower().translate(
    {ord(i): None for i in c.keys() if i not in letters}
)

bookTxt[:256]

In [ ]:
s = Counter(bookTxt)
s

That's the data we want to work with. Let's look at a barchart to show the relative frequencies visually

In [ ]:
plt.bar(*zip(*s.most_common()), width=.75, color='g')

This corrsponds with our intuition. The space ' '  is the most common character, then we have 'e', 't', 'a' ...

We will make a pandas DataFrame with this information as the first column. DataFrames are very flexible and intuitive objects you'll find in most programming languages when dealing with data analysis.

In [ ]:
holmesDF = pd.DataFrame.from_dict(s, 
                                  orient='index', 
                                  columns=['count']
                                 ).sort_values(by='count',ascending=False)
holmesDF.head()

Let's add a column with the frequency of each letter as a probability, sum up all of the values in the count column, then divide that column by the total to estimate the letter probability. This isn't actually adding new information to the dataFrame (the information content is identical to the existing 'count' column, but it is convenient so let's ignore that.

In [ ]:
holmesDF['rel_freq'] = holmesDF['count'] / holmesDF['count'].sum()
holmesDF.head(n=10)

This is _almost_ the same distribution when doing the same thing for English in general, but 's' and 'h' are transposed (maybe all those references to 'SHerlock'. The rel_freq values should sum to 1 (within rounding) and we can treat them as a probability distribution for generating "English" text.

In [ ]:
holmesDF['rel_freq'].sum()

## Modeling


### $0^{th}$ order model - Uniform probabilities

We'll dig into the numpy module to do the selection, specifically the `numpy.random` submodule. One trick I recommend here is to set a `random_seed`. This will make your results reproducible. Most of the time you are dealing with 'pseudo' random numbers.

In [ ]:
np.random.seed(645)
uniTxt = np.random.choice(holmesDF.index, size=10240)

In [ ]:
print(''.join(uniTxt[:2048]))

This isn't great, if you try to read through it you'll see that we have lots of relatively uncommon letters `z`, `j` etc. There are also lots of funny combinations of letters. Let's confirm this by creating the same histogram as before for this text.

In [ ]:
s = Counter(uniTxt)
plt.bar(*zip(*s.most_common()), width=.75, color='g')

### $1^{\textrm{st}}$ order model
We'll we can't complain, this is exactly what we asked for. Let's try the same thing but using the frequencies we calculated.

In [ ]:
frqTxt = np.random.choice(holmesDF.index, size=10240, p=holmesDF['rel_freq'])
print(''.join(frqTxt[:2048]))

Already that is starting to look better. If you really squint you might be able to convince yourself that this is some lost dialect of finnish or something. Let's double check the frequencies.

In [ ]:
s = Counter(frqTxt)
plt.bar(*zip(*s.most_common()), width=.75, color='g')

Next we could look at probabilities for pairs or triples of letters and select groups based on those

### Surprise

Consider some binary system, for example a fire alarm. The fire alarm is an information source with two states. The fire alarm is either on or it is off, telling us there is or isn't a fire. For the vast majority of the time the alarm is (hopefully!) off. Intuitively, the `on` state seems to tell us more than the off state, but we want to be able to quantify that. We can do that with a quantity called surprise which we take to be the log of the reciprocal of the probability. That's quite a mouthful so let's think about what we would want from a measure of "surprise" then explore this specific implementation.

If something has a very small probability (say p=0.001, then the reciprocal of that is very large (1/p = 1000), so as probabilities get smaller, this quantity gets larger - something we want from a measure of surprise; improbable things are surprising! If something has p=1, then it is certain to happen, you shouldn't be surprised at all, happily $\log_2(1) = 0$

In [ ]:
p = np.linspace(1e-6, 1, 100, endpoint=False)
plt.plot(p, np.log2(1/p))

For the fire example, say that the probability of a fire is $p=0.001$, so the probability of no fire is $q= 1-p =0.999$, then these have surprise
$$
  S_p = \log_2(1/0.001) \approx 9.9658\\
  S_q = \log_2(1/0.999) \approx 0.0014
$$
(units are bits)

### [Hartley-Shannon Entropy](https://en.wikipedia.org/wiki/Shannon%E2%80%93Hartley_theorem)

We can now calculate the surprise for letters in our sample text

In [ ]:
holmesDF['surprise'] = np.log2(1/ holmesDF.rel_freq)

In [ ]:
holmesDF.head()

In [ ]:
holmesDF.describe()

## $2^{\textrm{nd}}$ order model

Letters aren't independnent in english. A q is far more likely to be followed by a `u` than by any other letters, and there are lots of combinations which we would struggle to pronounce e.g. `hpkrl`. We need to consider the correlations between these letters. Each letter is going to be followed by something (until we get to the end of the text), so if take the letter a, the the sum of probabilitie over all letters should be 1, i.e. $\sum_{x\in{ab...z }}  p_A(x) = 1$. For each letter in our source text, we want to count all of the times the next letter is `a`, all of the times it is `b` etc. There are lots of ways to do this, but for me, I'm most comfortable in numpy so I'll turn the book text into an array of letters

In [ ]:
bookLetters = np.array(list(bookTxt))
bookLetters

In [ ]:
# This tells us whether each element is 'a' or not as a boolean
bookLetters == 'a'

The `nonzero` method translates the `True` values into index positions, so we have the indexes for each occurence of the letter `a` in the text. Adding 1 to these values gives us all of the letters which follow the `a`.

In [ ]:
nextLetter = bookLetters[np.nonzero(bookLetters == 'a')[0] + 1]
nextLetter

And we can count up the occurences for each of these

In [ ]:
np.unique(nextLetter, return_counts=True)

Try changing `a` to `q` above!

In the holmesDF dataframe, I would like to be able to access all of these values so we will add columns for every key in the index, we will 29 new columns with these counts.

In [ ]:
for character in letters:
    nextLetter = bookLetters[np.nonzero(bookLetters == character)[0] + 1]

Oh... I forgot that the last letter of the text isn't followed by anything! We can use numpy indexing to avoid the last character of bookLetters

In [ ]:
nlCounts = {}
for character in letters:
    nextLetter = bookLetters[np.nonzero(bookLetters[:-1] == character)[0] + 1]
    char, count = np.unique(nextLetter, return_counts=True)
    nlCounts[character] = dict(zip(char,count))


In [ ]:
nlCounts['a']

In [ ]:
for firstLetter in letters:
    for secondLetter in letters:
        if secondLetter in nlCounts[firstLetter]:
            holmesDF.loc[firstLetter, secondLetter] = nlCounts[firstLetter][secondLetter]
        else:
            holmesDF.loc[firstLetter, secondLetter] = 0.

holmesDF

We can translate these into probabilities if we normalize by the letter count

In [ ]:
# holmesDF.loc[:, letters].div(holmesDF['count'], axis='index')
holmesDF.loc[:, letters] = holmesDF.loc[:, letters].div(holmesDF['count'], axis='index')

In [ ]:
holmesDF

Now let's look at a heatmap of the letter pair probabilities

In [ ]:
holmesDF.loc[:, letters].sort_index()

In [ ]:
hDF = holmesDF.loc[:, letters].sort_index()
plt.pcolor(hDF)
plt.yticks(np.arange(0.5, len(hDF.index), 1), hDF.index)
plt.xticks(np.arange(0.5, len(hDF.columns), 1), hDF.columns)
plt.show()

This can be a little bit hard to interperate, but on the y-axis we have the first letter, scanning across each row the colours depict the probability of finding the letter in that column next. The bright yellow dot is corresponds to the `q`, `u` association. There are 413 occurrences of `q` in our text and every time, it is followed by a `u`, so the yellow corresponds to a probability of 1. We want our second order model to use these probabilities, so if a q is selected our model should _always_ spit out a `u` next.

Our model should work like this: For the first letter make a random choice from with probability `rel_freq`. Say we selected the letter 't'. Next we look at the row with index `t` in `holmesDF` we select the next letter according to the letter probabilities in that row. We keep iterating this second step to generate our sample text.

In [ ]:
firstLetter = np.random.choice(holmesDF.index, size=1, p=holmesDF['rel_freq'])
firstLetter

Given that we have selected the letter `r`, here are the probabilities for selecting any other char

In [ ]:
holmesDF.loc['r', letters]

In [ ]:
holmesDF.loc['r', letters].plot(kind='bar', width=.75, color='g')

So the character `e` is most likely to be selected next, followed by the space character, then `o` etc. Let's iterate this process

It will be handy to have each of our probability arrays in memory so let's build them from our dataframe

In [ ]:
letters

In [ ]:
letterP = {}
for letter in letters:
    lSeries = holmesDF.loc[letter, letters]
    letterP[letter] = (lSeries / lSeries.sum()).to_dict()

In [ ]:
secondOut = [firstLetter[0]]

for i in range(512):
    secondOut.append(
        np.random.choice(
            list(letterP[secondOut[-1]].keys()), 
            size=1, 
            p=list(letterP[secondOut[-1]].values())
        )[0])
    
print(''.join(secondOut))

`bile`, `rat`, `wean`, `wave`, ...